In [1]:
import requests
from lxml import html
import time


start_url = "https://www.immobiliare.it/66942457-Vendita-Bilocale-via-Giuseppe-Garibaldi-20-Loano.html"
urls_to_scrape = [start_url]
urls_scraped = []
urls_posts = []

# in this list are stored all the paths needed to parse on the homepage

post_paths = []

title = '//*[@id="sticky-contact-bottom"]/div[1]/h1/text()'
price = '//*[@id="sticky-contact-bottom"]/div[1]/div[1]/div[1]/ul[1]/li[1]/span/text()'
rooms = '//*[@id="sticky-contact-bottom"]/div[1]/div[1]/div[1]/ul[2]/li[1]/div[1]/span/text()'
bathrooms = '//*[@id="sticky-contact-bottom"]/div[1]/div[1]/div[1]/ul[2]/li[2]/div[1]/span/text()'
area = '//*[@id="sticky-contact-bottom"]/div[1]/div[1]/div[1]/ul[2]/li[3]/div[1]/span/text()'

post_Xpath = [title,price,rooms,bathrooms,area]

sleep_time = 2 # seconds

def main():
    
    # if the list of links to scrape is not empty
    if urls_to_scrape:
        
        # take the fist element of the list  
        url = urls_to_scrape[0]
        # open the url
        r = requests.get(url)
        # transform the pure text in HTML 
        tree = html.fromstring(r.text)
        print(tree)
        # for all the path in paths 
        post = []
        for path in post_Xpath:

            info = tree.xpath(path)
            post.append(info)

        print(post)
                
# here we initialize the script
main()

<Element html at 0x110b3f2c8>
[['Bilocale via Giuseppe Garibaldi 20, Loano'], ['€ 279.000'], ['2\xa0'], ['1\xa0'], ['53']]


In [9]:
paths = { 
            'title':'//*[@id="sticky-contact-bottom"]/div[1]/h1/text()',
            'price':'//*[@id="sticky-contact-bottom"]/div[1]/div[1]/div[1]/ul[1]/li[1]/span/text()',
            'rooms':'//*[@id="sticky-contact-bottom"]/div[1]/div[1]/div[1]/ul[2]/li[1]/div[1]/span/text()',
            'bathrooms':'//*[@id="sticky-contact-bottom"]/div[1]/div[1]/div[1]/ul[2]/li[2]/div[1]/span/text()',
            'area':'//*[@id="sticky-contact-bottom"]/div[1]/div[1]/div[1]/ul[2]/li[3]/div[1]/span/text()'
        }

def main():
    # if the list of links to scrape is not empty
    if urls_to_scrape:
        
        # take the fist element of the list  
        url = urls_to_scrape[0]
        # open the url
        r = requests.get(url)
        # transform the pure text in HTML 
        tree = html.fromstring(r.text)
        # for all the path in paths 
        post = []
        d = {}
        for key, value in paths.items():
            d.update({ key : tree.xpath(value)[0].replace('€','').replace('\xa0','').replace('.','').strip() })
            
        print(d)

main()       

{'title': 'Bilocale via Giuseppe Garibaldi 20, Loano', 'price': '279000', 'rooms': '2', 'bathrooms': '1', 'area': '53'}


# Create a Class that will handle this entire process for you

In [19]:
# our first definition of SinglePost class
class SinglePost():
    def __init__(self,url):
        self.url = url
        self.paths = { 
                      'title':'//*[@id="sticky-contact-bottom"]/div[1]/h1/text()',
                      'price':'//*[@id="sticky-contact-bottom"]/div[1]/div[1]/div[1]/ul[1]/li[1]/span/text()',
                      'rooms':'//*[@id="sticky-contact-bottom"]/div[1]/div[1]/div[1]/ul[2]/li[1]/div[1]/span/text()',
                      'bathrooms':'//*[@id="sticky-contact-bottom"]/div[1]/div[1]/div[1]/ul[2]/li[2]/div[1]/span/text()',
                      'area':'//*[@id="sticky-contact-bottom"]/div[1]/div[1]/div[1]/ul[2]/li[3]/div[1]/span/text()'
                     }
        
        # here we have some vars waiting to be assigned
        self.r = None
        self.tree = None
        self.data = {}
    
    # a function that is just waiting to be 
    def openUrl(self):
        self.r = requests.get(self.url)
        self.tree = html.fromstring(self.r.text)
    
    # check if the data if ok avoiding errors
    def controlData(self,data):
        f = None
        if data:
            f = data[0].replace('€','').replace('\xa0','').replace('.','')
        else:
            f = ''
        return f
    
    # extract the data from the xpath
    def getData(self):
        for key, value in self.paths.items():
            d = { key : self.controlData(self.tree.xpath(value)) }
            self.data.update(d)
    
    # just show our data 
    def show(self):
        print(self.data)
    
    # all things together
    def run(self):
        self.openUrl()
        self.getData()
        self.show()
            

In [20]:
import requests
from lxml import html
import time


start_url = "https://www.immobiliare.it/66942457-Vendita-Bilocale-via-Giuseppe-Garibaldi-20-Loano.html"
urls_to_scrape = [start_url]
urls_scraped = []
urls_posts = []

# in this list are stored all the paths needed to parse on the homepage
sleep_time = 2 # seconds

def main():
    
    # if the list of links to scrape is not empty
    if urls_to_scrape:
        
        # take the fist element of the list  
        url = urls_to_scrape[0]
        
        # open the url with our Class 
        s = SinglePost(url)
        s.run()
                
# here we initialize the script
main()

{'title': 'Bilocale via Giuseppe Garibaldi 20, Loano', 'price': ' 279000', 'rooms': '2', 'bathrooms': '1', 'area': '53'}


In [39]:
# import a bounch of stuff that allow us to play with sqlalchemy as ORM
from sqlalchemy import create_engine
from sqlalchemy import Column, DateTime, String, Text, Integer, ForeignKey, func, Boolean, update, or_, desc # some datatype and filters
from sqlalchemy.orm import relationship, backref, sessionmaker,session # session plus a little more
from sqlalchemy.ext.declarative import declarative_base

# we inicialize the declarative mode 
Base = declarative_base()

# we create a class that will map the columns in the db
class MyTable(Base):
    # table name
    __tablename__ = 'mytable'
    # all columns 
    id = Column('id',Integer, primary_key=True, autoincrement=True )
    address = Column('address',Text )
    price = Column('price',Text)
    rooms = Column('rooms',Text)
    bathrooms = Column('bathrooms',Text)
    area = Column('area',Text)
    url = Column('url',Text) # Text, can be also Integer, Boolean 

# were we gonna store the SQLite DB
db = create_engine("sqlite:///database.sqlite")
# it will plot the log of the dialog of the db
db.echo = True  # Try changing this to True 

# here we create the table if not present
session = sessionmaker()
session.configure(bind=db)
Base.metadata.create_all(db)

2018-07-02 15:26:09,351 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-02 15:26:09,353 INFO sqlalchemy.engine.base.Engine ()
2018-07-02 15:26:09,355 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-02 15:26:09,356 INFO sqlalchemy.engine.base.Engine ()
2018-07-02 15:26:09,359 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("mytable")
2018-07-02 15:26:09,360 INFO sqlalchemy.engine.base.Engine ()
2018-07-02 15:26:09,363 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE mytable (
	id INTEGER NOT NULL, 
	address TEXT, 
	price TEXT, 
	rooms TEXT, 
	bathrooms TEXT, 
	area TEXT, 
	url TEXT, 
	PRIMARY KEY (id)
)


2018-07-02 15:26:09,365 INFO sqlalchemy.engine.base.Engine ()
2018-07-02 15:26:09,369 INFO sqlalchemy.engine.base.Engine COMMIT


In [48]:
class SinglePost():
    def __init__(self,url):
        self.url = url
        self.paths = { 
                      'title':'//*[@id="sticky-contact-bottom"]/div[1]/h1/text()',
                      'price':'//*[@id="sticky-contact-bottom"]/div[1]/div[1]/div[1]/ul[1]/li[1]/span/text()',
                      'rooms':'//*[@id="sticky-contact-bottom"]/div[1]/div[1]/div[1]/ul[2]/li[1]/div[1]/span/text()',
                      'bathrooms':'//*[@id="sticky-contact-bottom"]/div[1]/div[1]/div[1]/ul[2]/li[2]/div[1]/span/text()',
                      'area':'//*[@id="sticky-contact-bottom"]/div[1]/div[1]/div[1]/ul[2]/li[3]/div[1]/span/text()'
                     }
        
        # here we have some vars waiting to be assigned
        self.r = None
        self.tree = None
        self.data = {}

    # a function that is just waiting to be 
    def openUrl(self):
        self.r = requests.get(self.url)
        self.tree = html.fromstring(self.r.text)  
    
    # check if the data if ok avoiding errors
    def controlData(self,data):
        f = None
        if data:
            f = data[0].replace('€','').replace('\xa0','').replace('.','')
        else:
            f = ''
        return f
    
    # extract the data from the xpath
    def getData(self):
        for key, value in self.paths.items():
            d = { key : self.controlData(self.tree.xpath(value)) }
            self.data.update(d)
            
    # just show our data 
    def show(self):
        print(self.data)
      
    def pushtoDB(self):
        s = session()
        post = MyTable( address = self.data['title'],
                      price = self.data['price'],
                      rooms = self.data['rooms'],
                      bathrooms = self.data['bathrooms'],
                      area = self.data['area'],
                      url = self.url )
        s.add(post)
        s.commit()
        
         
    def run(self):
        self.openUrl()
        self.getData()
        self.pushtoDB()
        self.show()

In [49]:
url = "https://www.immobiliare.it/66942457-Vendita-Bilocale-via-Giuseppe-Garibaldi-20-Loano.html"
single = SinglePost(url)
single.run()


2018-07-02 15:27:49,782 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-02 15:27:49,785 INFO sqlalchemy.engine.base.Engine INSERT INTO mytable (address, price, rooms, bathrooms, area, url) VALUES (?, ?, ?, ?, ?, ?)
2018-07-02 15:27:49,787 INFO sqlalchemy.engine.base.Engine ('Bilocale via Giuseppe Garibaldi 20, Loano', ' 279000', '2', '1', '53', 'https://www.immobiliare.it/66942457-Vendita-Bilocale-via-Giuseppe-Garibaldi-20-Loano.html')
2018-07-02 15:27:49,790 INFO sqlalchemy.engine.base.Engine COMMIT
{'title': 'Bilocale via Giuseppe Garibaldi 20, Loano', 'price': ' 279000', 'rooms': '2', 'bathrooms': '1', 'area': '53'}
